In [5]:
# インポート準備
import sys
sys.path.append('../')
from common.functions import *
import numpy as np

# 誤差逆伝播法

数値微分で勾配を計算するには時間がかかるため、誤差逆伝播法を使う  
理解するには、「数式」と「計算グラフ」の2つのアプローチが必要。

## 計算グラフ

計算をグラフ（データの形式）で表したもの

<img src="./スクリーンショット 2025-01-27 18.50.38.png" width="70%">

実際に簡単な問題を解いてみる。

```mermaid
graph LR
    A((りんご)) -->|100| B((✕))
    B -->|200| C((✕))
    C -->|220| D((代金))
    E((りんごの個数)) -->|2| B
    F((消費税)) -->|1.1| C
```

順伝播：左から右へ  
逆伝播：右から左へ

### 計算グラフの良いところ

- 局所的な計算ができる
  - 例）上のグラフで言うと、200円✕消費税の計算をするノードでは、なぜ200円になったかを意識しなくて良い
- 途中の計算結果を保持できる
- 逆伝播により微分を効率よく計算できる


上の例で逆伝播で微分を計算してみる

```mermaid
graph LR
    A[りんご] -->|x = 100| B[✕]
    B -->|t = 200| C[✕]
    C -->|220| D[代金L]
    E[りんごの個数] -->|2| B
    F[消費税] -->|1.1| C

    D -->|dL/dy = 1| C
    C -->|1 ✕ dL/dt = 1.1| B
    B -->|1.1 ✕ dt/dx = 2.2| A
```

図の右から左までの逆伝播で求められるもの  
= りんごの値段に関する支払金額の微分  
= りんごの少しの値上がりが、最終的な金額がどれくらい上がるか。

連鎖律（次の節で説明）を使って、合成関数の微分をだんだんに行っているイメージ

## 連鎖律

連鎖律を説明する前に合成関数を説明します。

### 合成関数
複数の関数で構成される関数  

例： $z=(x+y)^2$  

この式は下記のような2つの式で構成される。  
$z = t^2$  
$t = x + y$  

### 連鎖律

- 合成関数の微分の性質のこと
  - ある合成関数の微分　＝　ある合成関数を構成するそれぞれの関数の微分の積

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
$$

### 連鎖律を使って微分を求めてみる

$z=(x+y)^2$の微分を、連鎖律を使って求めてみます。  

$$
\frac{\partial z}{\partial t} = 2t
$$
$$
\frac{\partial t}{\partial x} = 1
$$

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t・1 = 2(x+y)
$$

### 連鎖律の計算を計算グラフで表してみる

```mermaid
graph LR
    A[ ] -->|x| B[＋]
    B -->|t| C[**2]
    C -->|z| D[ ]
    E[ ] -->|y| B

    D -->|dz/dz| C
    C -->|dz/dz ✕ dz/dt| B
    B -->|dz/dz ✕ dz/dt ✕ dt/dx| A
```

### まとめ
連鎖律には局所的な微分を伝達する原理がある。  
計算グラフの逆伝播とやっていることは同じ。

## 逆伝播

### 加算ノードの逆伝播
加算ノードを逆伝播するとどうなるのかについて考える。

$$z = x+y$$
この式を微分してみると
$$
\frac{\partial z}{\partial x} = 1
$$
$$
\frac{\partial z}{\partial y} = 1
$$

となる。  
これを計算グラフで書くと、

```mermaid
graph LR
    A[ ] -->|x| C[＋]
    B[ ] -->|y| C
    C -->|z| D[ ]
    D --> E[L]
```

```mermaid
graph RL
    A[L] --> B[ ]
    B -->|dL/dz| C[＋]
    C -->|dL/dz ✕ 1| D[ ]
    C -->|dL/dz ✕ 1| E[ ]
```

上の計算グラフから、dL/dzがそのまま逆伝播されているだけなのがわかる。  

実際の加算ノードでは下の計算グラフのように加算ノードの前後のノードで何らかの計算が行われているが、  
計算グラフ・連鎖律では局所的な微分を考えればよいので、何らかの計算は考慮しなくて良い。  

```mermaid
graph LR
    A[何らかの計算] -->|x| C[＋]
    B[何らかの計算] -->|y| C
    C -->|z| D[何らかの計算]
    D --> E[L]
```

そのため、加算ノードの逆伝播は1を乗算するだけ、つまり、逆伝播で入力された値をそのまま逆伝播するだけと考えて良い。

### 乗算ノードの逆伝播
乗算ノードを逆伝播するとどうなるのかについて考える。

$$z = xy$$
このような乗算の式を微分してみると
$$
\frac{\partial z}{\partial x} = y
$$
$$
\frac{\partial z}{\partial y} = x
$$

となる。  

つまり、乗算の逆伝播は、**上流の値　✕　順伝播の信号をひっくり返した値**　を下流に流す。  
これを計算グラフで書くと、

```mermaid
graph LR
    A[ ] -->|x| C[✕]
    B[ ] -->|y| C
    C -->|z| D[ ]
    D --> E[L]
```


```mermaid
graph RL
    A[L] --> B[ ]
    B -->|dL/dz| C[✕]
    C -->|dL/dz ✕ y| D[ ]
    C -->|dL/dz ✕ x| E[ ]
```

## 乗算ノードと加算ノードの実装

### レイヤ
レイヤとは機能の単位  
ニューラルネットワークでいう層のこと

乗算ノードと加算ノードをそれぞれ、乗算レイヤーと加算レイヤーとして実装する。  
レイヤーを一つのクラスで実装する

乗算レイヤの実装

In [6]:
# coding: utf-8

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

↑　`backward`関数の微分計算では、順伝播入力値をひっくり返して乗算する実装になっている。

乗算レイヤを使って、りんごの買い物を実装してみる

```mermaid
graph LR
    A[りんご] -->|x = 100| B[✕]
    B -->|t = 200| C[✕]
    C -->|220| D[代金L]
    E[りんごの個数] -->|2| B
    F[消費税] -->|1.1| C

    D -->|dL/dy = 1| C
    C -->|1 ✕ dL/dt = 1.1| B
    B -->|1.1 ✕ dt/dx = 2.2| A
```


In [7]:
# coding: utf-8

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dL/dx(りんごの値段が合計代金にどれくらい影響するか):", dapple)
print("dL/dy:", int(dapple_num))
print("dL/d消費税:", dtax)


price: 220
dL/dx(りんごの値段が合計代金にどれくらい影響するか): 2.2
dL/dy: 110
dL/d消費税: 200


↑　個数を乗算するレイヤーと、消費税を乗算するレイヤーとで入力値、出力値が異なるため  
　　個数と消費税のレイヤーそれぞれのインスタンスを作成している。  
　　（局所的な計算を保持できるのが計算グラフ、と前述したが、実際の実装でも保持できている）

加算レイヤ

In [8]:

class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

↑　今回は下のような計算グラフの構成となり、値を保持する必要がないため初期化はしていない。  
　　（乗算レイヤーで計算した値を入力して、計算計算結果を出力すれば良い）  
　　`pass`は何も行わないという命令



```mermaid
graph LR
    %% --- Apple part ---
    A[りんご単価] -->|100| B[✕]
    E[個数] -->|2| B
    B -->|りんご代200円| J[＋]
    B -->|2.2| A
    J -->|1.1| B

    %% --- Orange part ---
    G[みかん単価] -->|150| I[✕]
    H[個数] -->|3| I
    I -->|みかん代450円| J
    I -->|3.3| G

    %% --- Summation (Apple + Orange) ---
    J -->|小計 = 650| K[✕]
    K -->|1.1| J
    F[消費税] -->|1.1| K
    K -->|最終代金 = 715| D[代金 L]
```


In [9]:
# coding: utf-8

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## ここまでをまとめると

- 計算グラフは局所的な計算によって構成される
- 計算グラフの逆伝播によって各ノードの微分を求めることができる
- ノードをレイヤーとして実装し部品のように組み合わせた
  - 合成関数の微分の計算を簡単な実装で求めることができた

## 活性化関数レイヤの実装

前節ではりんごの値段の計算等を題材に計算グラフを実装したが、  
この節では計算グラフの考え方をニューラルネットワークに適用する。

活性化関数を実装する。

### ReLUレイヤ
ReLUの数式は下記で表される

$$
y(x) =
\begin{cases}
x & (x > 0) \\
0 & (x \leq 0) \\
\end{cases}
$$

これのxに関するyの微分を計算すると、

$$
\frac{\partial y}{\partial x}
=
\begin{cases}
1 & (x > 0) \\
0 & (x \leq 0) \\
\end{cases}
$$

この数式を計算グラフにあてはめると...  
順伝播の入力値が0より大きければ、逆伝播された値をそのまま下流に流す。  
順伝播の入力値が0以下であれば、０，つまり下流へ流れる値はなく、信号はそこでストップとなる。  

計算グラフで表すと以下のようになる。

```mermaid
graph LR
    A(( )) -->|x > 0| B((ReLU))
    B -->|y| C(( ))
    C -->|dL/dy| B
    B -->|dL/dy| A
```

```mermaid
graph LR
    A(( )) -->|x <= 0| B((ReLU))
    B -->|y| C(( ))
    C -->|dL/dy| B
    B -->|0| A
```

ReLUレイヤを実装すると、以下のコードになる。

In [21]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

### Sigmoidレイヤ
シグモイド関数

$$
y = \frac{1}{1+e^{-x}}
$$


In [22]:

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## Affineレイヤ
カメラライブラリ OpenCVのアフィン変換と

伝播するのが、スカラ値でなく行列になった。  


アフィンレイヤのバイアスに対する勾配 $\frac{\partial L}{\partial b}$  
バイアスの勾配は、出力の勾配 $\frac{\partial L}{\partial y}$ をそのまま伝播させるだけです。  
これは、バイアスが入力に対して直接加算されるためです。

具体的には、バイアスの勾配は以下のように計算されます。

$$ \frac{\partial L}{\partial b} = \sum_{i=1}^{N} \frac{\partial L}{\partial y_i} $$

In [11]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b

        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out


    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx

In [12]:
import numpy as np

x = np.array([
        [
            [1, 2],
            [3, 4]
        ],
        [
            [1, 2],
            [3, 4]
        ]
    ])
print(x.shape)  # 2*2*2
original_x_shape = x.shape
x = x.reshape(x.shape[0], -1)
# -1は残りの次元数を指定する
# つまり、8/2で、x = x.reshape(2, 4)　と同じ
x.shape

(2, 2, 2)


(2, 4)

### Soft-with-Lossレイヤ
3章で、softmax関数はニューラルネットワークで使わないと言っていたのは、  
あくまで、答えを一つだけ出したい推論時には、スコアの最大値をアフィンレイヤのスコア最大値に着目すればよいから必要ないという話だった。  
ニューラルネットワークの学習時には必要となる。

■交差エントロピー誤差を使う理由
softmax関数の損失関数として使うと、逆伝播がきれいな状態になるため（出力と正解の差）。

softmax-with-lossレイヤの実装

In [13]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

バッチサイズで割ることでデータ一個あたりの誤差が前レイヤへ伝播する  

使ってみる

In [14]:
# 入力データと教師データ
x = np.array([[0.3, 2.9, 4.0], [0.1, 0.2, 0.7]])
t = np.array([[0, 0, 1], [0, 1, 0]])

# SoftmaxWithLossクラスのインスタンスを作成
loss_layer = SoftmaxWithLoss()

# 順伝播
loss = loss_layer.forward(x, t)
print("Loss:", loss)

print('--------------------')
print(f'y-t:\n{loss_layer.y - loss_layer.t}')

# 逆伝播
dout = 1
dx = loss_layer.backward(dout)
print('--------------------')
print("dx = (y-t)/バッチサイズ:\n", dx)

Loss: 0.7868317613004097
--------------------
y-t:
[[ 0.01821127  0.24519181 -0.26340309]
 [ 0.25462853 -0.71859196  0.46396343]]
--------------------
dx = (y-t)/バッチサイズ:
 [[ 0.00910564  0.12259591 -0.13170154]
 [ 0.12731426 -0.35929598  0.23198171]]


バッチサイズで割るところを、あとで数式にしてみる

## 誤差逆伝播法の実装

In [15]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    # 認識結果を得る処理
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        # ワンホットエンコーディングであったら1次元のクラスラベルの配列に変換する
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # 重みパラメータに対する勾配を数値微分によって求める
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    # 勾配を誤差逆伝播法によって求める処理
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

レイヤを正しい順番に連結し呼び出すだけで、  
簡単にニューラルネットワークを構築できた。

### 誤差逆伝播法の勾配確認

In [16]:
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 各重みの絶対誤差の平均を求める
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:3.105146256561173e-10
b1:1.8790693390806336e-09
W2:4.3647966581394145e-09
b2:1.4010959351745678e-07


In [17]:
x = np.array([1, 2, 3])
print(x[:1])

[1]


### 誤差逆伝播法を使った学習

In [18]:
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
from common.functions import *  # Importing all functions from the common.functions module

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
print(f'train_size:{train_size}')
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 誤差逆伝播法によって勾配を求める
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)

    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

train_size:60000
0.10308333333333333 0.1023
0.9025 0.9057
0.9202166666666667 0.9208
0.93295 0.9322
0.9447166666666666 0.9427
0.9509 0.9481
0.9565666666666667 0.9524
0.9628333333333333 0.9588
0.9664166666666667 0.9605
0.9685833333333334 0.9626
0.9709833333333333 0.9636
0.9728 0.9665
0.97385 0.9674
0.9764333333333334 0.9692
0.9773166666666666 0.969
0.9783333333333334 0.9694
0.9797666666666667 0.9721
